<a href="https://colab.research.google.com/github/rtaglega53/pedropicapiedra/blob/master/CPA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


Adding all the possible necessary libraries



In [57]:
# librerias para manejo de datos y graficas
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from google.colab import files
import time
import warnings
# librerias para clustering
from sklearn import cluster, datasets, mixture, metrics
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_samples, silhouette_score
# librerias para busqueda de hiperparametros
from sklearn import metrics, preprocessing, tree
from sklearn.metrics import f1_score, make_scorer
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier as dt
from sklearn.model_selection import KFold
# librerias para PCA
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn import svm, metrics
warnings.filterwarnings("ignore")

Adding the file

In [166]:
# no se necesita cargar todo el tiempo
uploaded = files.upload()
df = pd.read_csv('customer_allocation_complete.csv')
df.head()

Saving customer_allocation_complete.csv to customer_allocation_complete (1).csv


,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Year,2021,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29,Unnamed: 30,Unnamed: 31,Unnamed: 32,Unnamed: 33,Unnamed: 34,Unnamed: 35,Unnamed: 36,Unnamed: 37,Unnamed: 38,Unnamed: 39,Unnamed: 40,Unnamed: 41,Unnamed: 42,Unnamed: 43,Unnamed: 44,Unnamed: 45,Unnamed: 46,Unnamed: 47,Unnamed: 48,Unnamed: 49,Unnamed: 50,Unnamed: 51,Unnamed: 52,Unnamed: 53,Unnamed: 54,Unnamed: 55,Unnamed: 56,Unnamed: 57,Unnamed: 58,Unnamed: 59,Unnamed: 60,Unnamed: 61,Unnamed: 62,Unnamed: 63,Unnamed: 64,Unnamed: 65,Unnamed: 66,Unnamed: 67,Unnamed: 68,Unnamed: 69,Unnamed: 70,Unnamed: 71,Unnamed: 72,Unnamed: 73,Unnamed: 74
0,NaN,NaN,NaN,NaN,Month,P02,NaN,NaN,NaN,NaN,NaN,NaN,P03,NaN,NaN,NaN,NaN,NaN,NaN,P04,NaN,NaN,NaN,NaN,NaN,NaN,P05,NaN,NaN,NaN,NaN,NaN,NaN,P06,NaN,NaN,NaN,NaN,NaN,NaN,P02,NaN,P03,NaN,P04,NaN,P05,NaN,P06,NaN,P02,NaN,NaN,NaN,NaN,P03,NaN,NaN,NaN,NaN,P04,NaN,NaN,NaN,NaN,P05,NaN,NaN,NaN,NaN,P06,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,Metrics,FC-6,FC-5,FC-4,FC-3,FC-2,FC-1,DLY,FC-6,FC-5,FC-4,FC-3,FC-2,FC-1,DLY,FC-6,FC-5,FC-4,FC-3,FC-2,FC-1,DLY,FC-6,FC-5,FC-4,FC-3,FC-2,FC-1,DLY,FC-6,FC-5,FC-4,FC-3,FC-2,FC-1,DLY,Score,Cluster_id,Score,Cluster_id,Score,Cluster_id,Score,Cluster_id,Score,Cluster_id,Baseline Qty (score based),Baseline Qty (weighted average),Baseline Qty (Exponential smoothing 1),Baseline Qty (Exponential smoothing 2),Baseline Qty (Exponential smoothing 3),Baseline Qty (score based),Baseline Qty (weighted average),Baseline Qty (Exponential smoothing 1),Baseline Qty (Exponential smoothing 2),Baseline Qty (Exponential smoothing 3),Baseline Qty (score based),Baseline Qty (weighted average),Baseline Qty (Exponential smoothing 1),Baseline Qty (Exponential smoothing 2),Baseline Qty (Exponential smoothing 3),Baseline Qty (score based),Baseline Qty (weighted average),Baseline Qty (Exponential smoothing 1),Baseline Qty (Exponential smoothing 2),Baseline Qty (Exponential smoothing 3),Baseline Qty (score based),Baseline Qty (weighted average),Baseline Qty (Exponential smoothing 1),Baseline Qty (Exponential smoothing 2),Baseline Qty (Exponential smoothing 3)
2,MaterialNumber,Market,Country,CustomerFCGCode,CustomerFCGName,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,7981020031,Market America,UNITED STATES,C4056,VERIZON WIRELINE-USA,336,0,0,0,0,0,0,336,252,210,0,42,42,42,378,378,420,420,0,84,84,336,420,378,378,378,0,0,378,462,462,420,378,378,378,NaN,17,-3.9,20,-11.6,21,NaN,21,20.7,21,0,0,0,0,0,42,13,42,43,162,84,8,80,88,397,0,0,52,91,373,378,424,379,381,449
4,600044-641-005,Market America,UNITED STATES,C4394,CENTURYLINK-USA,0,0,0,0,0,0,0,50,50,50,50,50,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,50,50,50,50,50,0,0,NaN,NaN,NaN,23,NaN,NaN,NaN,NaN,NaN,23,0,0,0,0,0,0,0,7,12,49,0,0,0,0,0,0,0,0,0,0,0,0,7,12,49


In [18]:
df_base = df.copy()
df = df_base.copy()

Clean the data and save months and cycles in a separated variables

In [168]:
# utilizamos metodo copy para poder jugar con los valors sin tener que cargar el archivo
months = df.iloc[0]
cycles = df.iloc[1]
df_copy = df.drop(columns=['Unnamed: 0', 'Unnamed: 1', 'Unnamed: 2', 'Unnamed: 3']).copy()
df_copy = df_copy.drop(index=[2])
columns_names = df_copy.columns
df_copy.columns = df_copy.iloc[1]
df_copy = df_copy.drop(columns=['DLY'])
df_copy.columns.name = None
months = df_copy.iloc[0]
cycles = df_copy.iloc[1]
df_copy = df_copy.drop(index=[1])
df_copy.head()


,Metrics,FC-6,FC-5,FC-4,FC-3,FC-2,FC-1,FC-6,FC-5,FC-4,FC-3,FC-2,FC-1,FC-6,FC-5,FC-4,FC-3,FC-2,FC-1,FC-6,FC-5,FC-4,FC-3,FC-2,FC-1,FC-6,FC-5,FC-4,FC-3,FC-2,FC-1,Score,Cluster_id,Score,Cluster_id,Score,Cluster_id,Score,Cluster_id,Score,Cluster_id,Baseline Qty (score based),Baseline Qty (weighted average),Baseline Qty (Exponential smoothing 1),Baseline Qty (Exponential smoothing 2),Baseline Qty (Exponential smoothing 3),Baseline Qty (score based),Baseline Qty (weighted average),Baseline Qty (Exponential smoothing 1),Baseline Qty (Exponential smoothing 2),Baseline Qty (Exponential smoothing 3),Baseline Qty (score based),Baseline Qty (weighted average),Baseline Qty (Exponential smoothing 1),Baseline Qty (Exponential smoothing 2),Baseline Qty (Exponential smoothing 3),Baseline Qty (score based),Baseline Qty (weighted average),Baseline Qty (Exponential smoothing 1),Baseline Qty (Exponential smoothing 2),Baseline Qty (Exponential smoothing 3),Baseline Qty (score based),Baseline Qty (weighted average),Baseline Qty (Exponential smoothing 1),Baseline Qty (Exponential smoothing 2),Baseline Qty (Exponential smoothing 3)
0,Month,P02,NaN,NaN,NaN,NaN,NaN,P03,NaN,NaN,NaN,NaN,NaN,P04,NaN,NaN,NaN,NaN,NaN,P05,NaN,NaN,NaN,NaN,NaN,P06,NaN,NaN,NaN,NaN,NaN,P02,NaN,P03,NaN,P04,NaN,P05,NaN,P06,NaN,P02,NaN,NaN,NaN,NaN,P03,NaN,NaN,NaN,NaN,P04,NaN,NaN,NaN,NaN,P05,NaN,NaN,NaN,NaN,P06,NaN,NaN,NaN,NaN
3,VERIZON WIRELINE-USA,336,0,0,0,0,0,336,252,210,0,42,42,378,378,420,420,0,84,336,420,378,378,378,0,378,462,462,420,378,378,NaN,17,-3.9,20,-11.6,21,NaN,21,20.7,21,0,0,0,0,0,42,13,42,43,162,84,8,80,88,397,0,0,52,91,373,378,424,379,381,449
4,CENTURYLINK-USA,0,0,0,0,0,0,50,50,50,50,50,0,0,0,0,0,0,0,0,0,0,0,0,0,50,50,50,50,50,0,NaN,NaN,NaN,23,NaN,NaN,NaN,NaN,NaN,23,0,0,0,0,0,0,0,7,12,49,0,0,0,0,0,0,0,0,0,0,0,0,7,12,49
5,CENTURYLINK-USA,250,250,250,100,0,0,250,250,250,100,100,0,250,250,250,100,0,0,250,250,250,100,100,0,250,250,250,100,0,0,NaN,20,NaN,20,NaN,20,NaN,20,NaN,20,0,0,2,8,208,0,0,14,26,212,0,0,2,8,208,0,0,14,26,212,0,0,2,8,208
6,ORANGE-FRA,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,30,30,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,-3.9,7,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,9,29,28,2,0,0,0,0,0,0,0,0,0,0


data set is transposed for the analysis


In [169]:
df_copy = df_copy.transpose()

df_copy.head()


#df_copy.describe(include=np.object)

,0,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,...,6696,6697,6698,6699,6700,6701,6702,6703,6704,6705,6706,6707,6708,6709,6710,6711,6712,6713,6714,6715,6716,6717,6718,6719,6720,6721,6722,6723,6724,6725,6726,6727,6728,6729,6730,6731,6732,6733,6734,6735
Metrics,Month,VERIZON WIRELINE-USA,CENTURYLINK-USA,CENTURYLINK-USA,ORANGE-FRA,ORANGE-ESP,VODAFONE-ESP,SWISSCOM-CHE,SINGTEL-PHL,DEUTSCHE TELEKOM-AUT,TDC-DNK,ORANGE-FRA,VODAFONE-ITA,TT-GO-MLT,EMEA NATIONAL SP-PRT,ORANGE-ESP,VODAFONE-ESP,VODAFONE-NZL,VEON-WIND-ITA,EMEA NATIONAL SP-PRT,Other Customers in NIGERIA,OOREDOO-OMN,SPRINT NEXTEL-USA,UTILITIES-USA,OTLE-USA,SHENTEL-USA,REGIONALS-USA,TMOBILE-USA,TMOBILE-USA,RAKUTEN-JPN,MTC-SAU,TELUS-CAN,VHA-AUS,VODAFONE-NZL,SINGTEL-PHL,IBG GCHN - DITO,CHUNGHWA TELECOM-TWN,ASIA PACIFIC TELECOM-TWN,FAR EASTONE TELECOM-TW,TAIWAN MOBILE-TWN,...,CHARTER-USA,Vinaphone-VNM,ORANGE-FRA,CNT-ECU,TELUS-CAN,UTILITIES-CAN,SASKTEL-CAN,COGECO ENTITY-CAN,UTILITIES-USA,REGIONALS-USA,NORTHWEST FIBER-USA,CINCINNATI BELL TELEPHONE COMPANY LLC.,HOTWIRE COMMUNICATIONS LTD,METRONET,BELL CANADA-CAN,GOV & CITIES-USA,OTLE-USA,Two Degrees-NZL,END CUSTOMERS HOTH IN SPAIN,VM PUBLIC SECTOR-INDIRECT-MEX,TRANSPORT&LOGISTICS OTH-FRA,GOV AND CITIES CHAN-FRA,OTHER LE-CHAN-DIST-FRA,GOV AND CITIES-CHAN-DIST-FRA,OTHER SERV. PROV.-AXIONE-FRA,Nuvias Deutschland-DEU,END CUSTOMERS TXLE IN TURKEY,ORANGE-FRA,GOVERNMENT AND CITIES-SGP,OTHER CUSTOMERS IN OMAN,END CUSTOMERS TXLE IN AUSTRALIA,GOVERNMENT AND CITIES-SGP,OTHER SERV. PROVIDERS-IDN,Bali Tower-IDN,END CUSTOMERS HOTH IN SPAIN,GOV AND CITIES-ESP,OTHER SERV. PROVIDERS-UKR,OTHER LE-CHAN-DEU,SP-CABLE-IND,TRANSPORTATION-BRA
FC-6,P02,336,0,250,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,500,0,2000,0,0,0,0,1000,0,0,0,0,0,...,10,0,0,0,8000,0,20,0,0,200,0,0,0,50,100,0,0,0,8,0,4,0,0,163,0,0,0,0,0,47,0,0,0,0,5,0,0,0,0,5
FC-5,NaN,0,0,250,0,0,0,0,0,0,0,0,0,0,0,0,0,800,0,0,0,0,200,0,0,0,500,500,0,2000,10000,0,0,0,1000,0,50,0,0,50,...,12,0,11,0,8000,0,20,0,0,200,0,0,0,50,100,0,0,0,0,120,4,0,0,0,0,0,0,0,0,47,0,0,0,0,0,0,0,0,0,0
FC-4,NaN,0,0,250,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,200,0,0,0,500,2352,0,0,10000,0,0,0,1000,0,0,0,0,0,...,2,0,11,0,7000,0,20,0,250,200,0,0,0,50,200,10,0,0,0,0,0,0,0,0,0,0,538,0,0,0,0,0,0,1,0,0,0,0,0,0
FC-3,NaN,0,0,100,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4000,0,0,0,0,0,0,2352,0,0,10000,0,0,0,100,0,0,0,0,0,...,0,0,11,20,11000,0,0,0,250,200,0,0,2500,50,200,10,12,0,0,0,0,0,0,0,0,0,538,6,0,0,0,0,0,0,0,0,37,0,0,0


In [171]:
df_copy.columns = df_copy.iloc[0]
# df_copy = df_copy.drop()
df_copy.head()

Metrics,Month,VERIZON WIRELINE-USA,CENTURYLINK-USA,CENTURYLINK-USA,ORANGE-FRA,ORANGE-ESP,VODAFONE-ESP,SWISSCOM-CHE,SINGTEL-PHL,DEUTSCHE TELEKOM-AUT,TDC-DNK,ORANGE-FRA,VODAFONE-ITA,TT-GO-MLT,EMEA NATIONAL SP-PRT,ORANGE-ESP,VODAFONE-ESP,VODAFONE-NZL,VEON-WIND-ITA,EMEA NATIONAL SP-PRT,Other Customers in NIGERIA,OOREDOO-OMN,SPRINT NEXTEL-USA,UTILITIES-USA,OTLE-USA,SHENTEL-USA,REGIONALS-USA,TMOBILE-USA,TMOBILE-USA,RAKUTEN-JPN,MTC-SAU,TELUS-CAN,VHA-AUS,VODAFONE-NZL,SINGTEL-PHL,IBG GCHN - DITO,CHUNGHWA TELECOM-TWN,ASIA PACIFIC TELECOM-TWN,FAR EASTONE TELECOM-TW,TAIWAN MOBILE-TWN,...,CHARTER-USA,Vinaphone-VNM,ORANGE-FRA,CNT-ECU,TELUS-CAN,UTILITIES-CAN,SASKTEL-CAN,COGECO ENTITY-CAN,UTILITIES-USA,REGIONALS-USA,NORTHWEST FIBER-USA,CINCINNATI BELL TELEPHONE COMPANY LLC.,HOTWIRE COMMUNICATIONS LTD,METRONET,BELL CANADA-CAN,GOV & CITIES-USA,OTLE-USA,Two Degrees-NZL,END CUSTOMERS HOTH IN SPAIN,VM PUBLIC SECTOR-INDIRECT-MEX,TRANSPORT&LOGISTICS OTH-FRA,GOV AND CITIES CHAN-FRA,OTHER LE-CHAN-DIST-FRA,GOV AND CITIES-CHAN-DIST-FRA,OTHER SERV. PROV.-AXIONE-FRA,Nuvias Deutschland-DEU,END CUSTOMERS TXLE IN TURKEY,ORANGE-FRA,GOVERNMENT AND CITIES-SGP,OTHER CUSTOMERS IN OMAN,END CUSTOMERS TXLE IN AUSTRALIA,GOVERNMENT AND CITIES-SGP,OTHER SERV. PROVIDERS-IDN,Bali Tower-IDN,END CUSTOMERS HOTH IN SPAIN,GOV AND CITIES-ESP,OTHER SERV. PROVIDERS-UKR,OTHER LE-CHAN-DEU,SP-CABLE-IND,TRANSPORTATION-BRA
Metrics,Month,VERIZON WIRELINE-USA,CENTURYLINK-USA,CENTURYLINK-USA,ORANGE-FRA,ORANGE-ESP,VODAFONE-ESP,SWISSCOM-CHE,SINGTEL-PHL,DEUTSCHE TELEKOM-AUT,TDC-DNK,ORANGE-FRA,VODAFONE-ITA,TT-GO-MLT,EMEA NATIONAL SP-PRT,ORANGE-ESP,VODAFONE-ESP,VODAFONE-NZL,VEON-WIND-ITA,EMEA NATIONAL SP-PRT,Other Customers in NIGERIA,OOREDOO-OMN,SPRINT NEXTEL-USA,UTILITIES-USA,OTLE-USA,SHENTEL-USA,REGIONALS-USA,TMOBILE-USA,TMOBILE-USA,RAKUTEN-JPN,MTC-SAU,TELUS-CAN,VHA-AUS,VODAFONE-NZL,SINGTEL-PHL,IBG GCHN - DITO,CHUNGHWA TELECOM-TWN,ASIA PACIFIC TELECOM-TWN,FAR EASTONE TELECOM-TW,TAIWAN MOBILE-TWN,...,CHARTER-USA,Vinaphone-VNM,ORANGE-FRA,CNT-ECU,TELUS-CAN,UTILITIES-CAN,SASKTEL-CAN,COGECO ENTITY-CAN,UTILITIES-USA,REGIONALS-USA,NORTHWEST FIBER-USA,CINCINNATI BELL TELEPHONE COMPANY LLC.,HOTWIRE COMMUNICATIONS LTD,METRONET,BELL CANADA-CAN,GOV & CITIES-USA,OTLE-USA,Two Degrees-NZL,END CUSTOMERS HOTH IN SPAIN,VM PUBLIC SECTOR-INDIRECT-MEX,TRANSPORT&LOGISTICS OTH-FRA,GOV AND CITIES CHAN-FRA,OTHER LE-CHAN-DIST-FRA,GOV AND CITIES-CHAN-DIST-FRA,OTHER SERV. PROV.-AXIONE-FRA,Nuvias Deutschland-DEU,END CUSTOMERS TXLE IN TURKEY,ORANGE-FRA,GOVERNMENT AND CITIES-SGP,OTHER CUSTOMERS IN OMAN,END CUSTOMERS TXLE IN AUSTRALIA,GOVERNMENT AND CITIES-SGP,OTHER SERV. PROVIDERS-IDN,Bali Tower-IDN,END CUSTOMERS HOTH IN SPAIN,GOV AND CITIES-ESP,OTHER SERV. PROVIDERS-UKR,OTHER LE-CHAN-DEU,SP-CABLE-IND,TRANSPORTATION-BRA
FC-6,P02,336,0,250,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,500,0,2000,0,0,0,0,1000,0,0,0,0,0,...,10,0,0,0,8000,0,20,0,0,200,0,0,0,50,100,0,0,0,8,0,4,0,0,163,0,0,0,0,0,47,0,0,0,0,5,0,0,0,0,5
FC-5,NaN,0,0,250,0,0,0,0,0,0,0,0,0,0,0,0,0,800,0,0,0,0,200,0,0,0,500,500,0,2000,10000,0,0,0,1000,0,50,0,0,50,...,12,0,11,0,8000,0,20,0,0,200,0,0,0,50,100,0,0,0,0,120,4,0,0,0,0,0,0,0,0,47,0,0,0,0,0,0,0,0,0,0
FC-4,NaN,0,0,250,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,200,0,0,0,500,2352,0,0,10000,0,0,0,1000,0,0,0,0,0,...,2,0,11,0,7000,0,20,0,250,200,0,0,0,50,200,10,0,0,0,0,0,0,0,0,0,0,538,0,0,0,0,0,0,1,0,0,0,0,0,0
FC-3,NaN,0,0,100,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4000,0,0,0,0,0,0,2352,0,0,10000,0,0,0,100,0,0,0,0,0,...,0,0,11,20,11000,0,0,0,250,200,0,0,2500,50,200,10,12,0,0,0,0,0,0,0,0,0,538,6,0,0,0,0,0,0,0,0,37,0,0,0


Declare month of the quarter

In [82]:
months_range = ['CM', '+1M', '+2M', '+3M']
month_base = 0
if len(months_range) == 4:
    month_base = 0
elif len(months_range) == 5:
  month_base = 1
elif len(months_range) == 6:
  month_base = 2
# print(len(months_range), month_base)
df_copy.shape
df_copy.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6734 entries, 0 to 6735
Data columns (total 66 columns):
 #   Column                                  Non-Null Count  Dtype 
---  ------                                  --------------  ----- 
 0   Metrics                                 6734 non-null   object
 1   FC-6                                    6734 non-null   object
 2   FC-5                                    6733 non-null   object
 3   FC-4                                    6733 non-null   object
 4   FC-3                                    6733 non-null   object
 5   FC-2                                    6733 non-null   object
 6   FC-1                                    6733 non-null   object
 7   FC-6                                    6734 non-null   object
 8   FC-5                                    6733 non-null   object
 9   FC-4                                    6733 non-null   object
 10  FC-3                                    6733 non-null   object
 11  FC-2